Use this notebook to alter column values in the scraped data in order to create a diffed version of the data that can be used to test the updating functionality.

In [66]:
import numpy as np
import pandas as pd

np.random.seed(618)

In [67]:
xl = pd.ExcelFile('../Data/Complaint_Data_Scraped.xlsx')
sheetnames = xl.sheet_names
sheetnames

['Panel complaint data scraped',
 'EIB complaint data scraped',
 'MICI complaint data scraped',
 'CAO complaint data scraped',
 'ADB complaint data scraped']

In [68]:
dfs = {}
for name in sheetnames:
    df = xl.parse(name)
    dfs[name.split()[0]] = df
    print(name.split()[0],df.shape)
    print(df.columns,'\n','*'*40)

Panel (122, 31)
Index(['IAM', 'Year', 'Country', 'Project', 'ID', 'IAM ID', 'Filer(s)',
       'Environmental Category', 'Project Company', 'Project Number',
       'Related Project Number', 'Project Type', 'Financial Institution',
       'Project Loan Amount', 'Sector', 'Issues', 'Complaint Status',
       'Filing Date', 'Registration Start Date', 'Registration End Date',
       'Eligibility Start Date', 'Eligibility End Date',
       'Dispute Resolution Start Date', 'Dispute Resolution End Date',
       'Compliance Review Start Date', 'Compliance Review End Date',
       'Monitoring Start Date', 'Monitoring End Date', 'Date Closed',
       'Documents', 'Hyperlink'],
      dtype='object') 
 ****************************************
EIB (197, 31)
Index(['IAM', 'Year', 'Country', 'Project', 'ID', 'IAM ID', 'Filer(s)',
       'Environmental Category', 'Project Company', 'Project Number',
       'Related Project Number', 'Project Type', 'Financial Institution',
       'Project Loan Amount'

Below I select columns that would likely get updates, leaving the ADB data alone as a "no change" benchmark.

In [77]:
to_permute = {'Panel': ['Filing Date', 'Date Closed'], 
              'EIB': ['Dispute Resolution End Date', 'Documents'],
              'MICI': ['Compliance Review Start Date', 'Date Closed'],
              'CAO': ['Monitoring Start Date', 'Monitoring End Date']}

In [78]:
for k, v in to_permute.items():
    for x in v:
        print(k, x)

Panel Filing Date
Panel Date Closed
EIB Dispute Resolution End Date
EIB Documents
MICI Compliance Review Start Date
MICI Date Closed
CAO Monitoring Start Date
CAO Monitoring End Date


In [60]:
dfs['Panel']['Filing Date'].head()

0    Completed
1    Completed
2          NaN
3          NaN
4    Completed
Name: Filing Date, dtype: object

In [79]:
for df, cols in to_permute.items():
    for col in cols:
        print('Permuting {} in {}'.format(col, df))
        dfs[df][col] = np.random.permutation(dfs[df][col])

Permuting Filing Date in Panel
Permuting Date Closed in Panel
Permuting Dispute Resolution End Date in EIB
Permuting Documents in EIB
Permuting Compliance Review Start Date in MICI
Permuting Date Closed in MICI
Permuting Monitoring Start Date in CAO
Permuting Monitoring End Date in CAO


In [80]:
dfs['Panel']['Filing Date'].head()

0          NaN
1          NaN
2    Completed
3          NaN
4          NaN
Name: Filing Date, dtype: object

In [81]:
writer = pd.ExcelWriter('Complaint_Data_Scraped_diff.xlsx', engine='xlsxwriter',
                        options={'strings_to_urls': False})

dfs['Panel'].to_excel(writer, sheet_name='Panel complaint data scraped', index=False)
dfs['EIB'].to_excel(writer, sheet_name='EIB complaint data scraped', index=False)
dfs['MICI'].to_excel(writer, sheet_name='MICI complaint data scraped', index=False)
dfs['CAO'].to_excel(writer, sheet_name='CAO complaint data scraped', index=False)
dfs['ADB'].to_excel(writer, sheet_name='ADB complaint data scraped', index=False)

writer.save()